# 3. Tìm hiểu kỹ thuật sử dụng các mô hình mạng CNN kinh điển như VGG, ResNet, DenseNet với framework Keras, thử áp dụng cho các bài toán nhận diện cảm xúc, phân loại phương tiện giao thông:
https://keras.io/applications/


# Phân loại phương tiện giao thông 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/Colab Notebooks/L12

/content/drive/My Drive/Colab Notebooks/L12


In [0]:
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

Using TensorFlow backend.


## Hàm đọc dữ liệu (training set and test set)
Bộ dữ liệu của bài toán được đặt ở thư mục './Dataset/vehicle'. Trong thư mục này ta có 2 thư mục con: Training, Testing tương ứng với dữ liệu để huấn luyện mô hình và dữ liệu để kiểm tra độ chính xác của mô hình. 

In [0]:
import numpy as np
import os
import cv2
import pickle

def prepair_data(path='./Dataset/vehicle', img_shape=(32, 32)):
    TRAINING_DATA_PATH = os.path.join(path, 'Training')
    TESTING_DATA_PATH = os.path.join(path, 'Testing')

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    label_id = 0
    
    num_classes = len(os.listdir(TRAINING_DATA_PATH))
    for label in os.listdir(TRAINING_DATA_PATH):

        # Read training data
        for img_file in os.listdir(os.path.join(TRAINING_DATA_PATH, label)):
            print(img_file)
            img = cv2.imread(os.path.join(TRAINING_DATA_PATH, label, img_file))
            img = cv2.resize(img, img_shape)
            x_train.append(img)

            y = np.zeros(num_classes)
            y[label_id] = 1
            y_train.append(y)

        # Read testing data
        for img_file in os.listdir(os.path.join(TESTING_DATA_PATH, label)):
            print(img_file)
            img = cv2.imread(os.path.join(TESTING_DATA_PATH, label, img_file))
            img = cv2.resize(img, img_shape)
            x_test.append(img)

            y = np.zeros(num_classes)
            y[label_id] = 1
            y_test.append(y)

        label_id += 1

    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

Đọc dữ liệu

In [0]:
dataset=()
filename=os.path.join('Dataset','vehicle.pkl')
if not os.path.isfile(filename):
  x_train, y_train, x_test, y_test = prepair_data(path='./Dataset/vehicle', img_shape=(32, 32))
  data_set=(x_train,y_train,x_test,y_test)
  pickle.dump(data_set,open(filename,'wb'))
else:
  x_train, y_train, x_test, y_test=pickle.load(open(filename,'rb'))

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1672, 32, 32, 3)
(1672, 5)
(183, 32, 32, 3)
(183, 5)


# Kĩ thuật fine-tune sử dụng các mô hình đã được huấn luyện sẵn
Keras cung cấp các mô hình kinh điển cho tập dữ liệu ImageNet với 1000 classes.   
Vì các mô hinh này là các bộ phân loại mạnh, nên đặc trưng trích xuất từ các mô hình này ( trước tầng fully-connected) là rất tốt và cho khả năng trích xuất đặc trưng cao. Xây dựng mô hình phân loại phương tiện giao thông trong đó tận dụng bộ trích xuất  các đặc trưng  ( từ các lớp convolution, pooling,..) của các mô hình kinh điển.

In [0]:
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.models import Model


## ResNet50 

In [0]:
weight_decay=1e-3
num_classes=5
base_model=ResNet50(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=num_classes)

x=base_model.output

x = Flatten()(x)

x = Dense(units=512, activation='relu', kernel_regularizer=l2(weight_decay))(x)

# Layer 4
x = Dense(units=64, activation='relu', kernel_regularizer=l2(weight_decay))(x)

# Layer 5
output = Dense(units=num_classes, activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=output)
model.summary()




W0630 10:51:33.464083 140201657169792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0630 10:51:33.503628 140201657169792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0630 10:51:33.515573 140201657169792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0630 10:51:33.551594 140201657169792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0630 10:51:33.552620

94658560/94653016 [==============================] - 1s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________

### Đóng băng toàn bộ weight ở các tầng  convolutionn. Training mô hình (các tầng fully-connected) qua một vài epochs 

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
from keras.optimizers import SGD
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

W0630 10:51:46.143952 140201657169792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# train the model on the new data for a few epochs
model.fit(x_train,y_train,epochs=20,batch_size=x_train.shape[0])

W0630 10:51:52.766646 140201657169792 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
1672/1672 [==============================] - 4s 2ms/step - loss: 3.1499 - acc: 0.2010
Epoch 2/20
1672/1672 [==============================] - 0s 94us/step - loss: 2.9970 - acc: 0.3559
Epoch 3/20
1672/1672 [==============================] - 0s 93us/step - loss: 2.0103 - acc: 0.6262
Epoch 4/20
1672/1672 [==============================] - 0s 92us/step - loss: 1.7067 - acc: 0.7249
Epoch 5/20
1672/1672 [==============================] - 0s 93us/step - loss: 1.5749 - acc: 0.7889
Epoch 6/20
1672/1672 [==============================] - 0s 93us/step - loss: 1.4515 - acc: 0.8421
Epoch 7/20
1672/1672 [==============================] - 0s 93us/step - loss: 1.3640 - acc: 0.8923
Epoch 8/20
1672/1672 [==============================] - 0s 93us/step - loss: 1.2749 - acc: 0.9199
Epoch 9/20
1672/1672 [==============================] - 0s 93us/step - loss: 1.1970 - acc: 0.9438
Epoch 10/20
1672/1672 [==============================] - 0s 93us/step - loss: 1.1404 - acc: 0.9569
Epoch 11/20
1672/167

In [0]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 pool1_pad
6 max_pooling2d_1
7 res2a_branch2a
8 bn2a_branch2a
9 activation_2
10 res2a_branch2b
11 bn2a_branch2b
12 activation_3
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add_1
18 activation_4
19 res2b_branch2a
20 bn2b_branch2a
21 activation_5
22 res2b_branch2b
23 bn2b_branch2b
24 activation_6
25 res2b_branch2c
26 bn2b_branch2c
27 add_2
28 activation_7
29 res2c_branch2a
30 bn2c_branch2a
31 activation_8
32 res2c_branch2b
33 bn2c_branch2b
34 activation_9
35 res2c_branch2c
36 bn2c_branch2c
37 add_3
38 activation_10
39 res3a_branch2a
40 bn3a_branch2a
41 activation_11
42 res3a_branch2b
43 bn3a_branch2b
44 activation_12
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_4
50 activation_13
51 res3b_branch2a
52 bn3b_branch2a
53 activation_14
54 res3b_branch2b
55 bn3b_branch2b
56 activation_15
57 res3b_branch2c
58 bn3b_branch2c
59 add_5
60 activation_16
61 res3c_branch2a
62 bn3c_branch2a

## Sau khi training các tầng fully-connected , thực hiện training toàn bộ mô hình.
Vì các tầng convolution là một bộ trích xuất đặc trưng mạnh, ta hi vọng giữ lại trọng số của các tầng này nhiều nhất có thể.  
Vì vậy, learning rate được đặt là 0.001 (nhỏ)

In [0]:
for layer in base_model.layers:
    layer.trainable = True

In [0]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit(x_train,y_train,batch_size=64,epochs=100)

Epoch 1/100
1672/1672 [==============================] - 10s 6ms/step - loss: 1.1554 - acc: 0.9211
Epoch 2/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0569 - acc: 0.9545
Epoch 3/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0349 - acc: 0.9659
Epoch 4/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0282 - acc: 0.9593
Epoch 5/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0035 - acc: 0.9749
Epoch 6/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0002 - acc: 0.9749
Epoch 7/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0031 - acc: 0.9749
Epoch 8/100
1672/1672 [==============================] - 2s 1ms/step - loss: 0.9969 - acc: 0.9785
Epoch 9/100
1672/1672 [==============================] - 2s 1ms/step - loss: 1.0003 - acc: 0.9737
Epoch 10/100
1672/1672 [==============================] - 2s 1ms/step - loss: 0.9797 - acc: 0.9839
Epoch 11/100
1672/

Test model đã huấn luyện với dữ liệu test:

In [0]:
model.evaluate(x_test, y_test)

183/183 [==============================] - 1s 6ms/step


[1.3094541296932867, 0.9016393432851697]

## VGG16 kết hợp data augmentation để tạo thêm dữ liệu 
Các bước tương tự như trên



In [0]:
from keras.applications.vgg16 import VGG16
from keras import backend as K
from keras.models import Model


In [0]:
weight_decay=1e-3
num_classes=5
base_model=VGG16(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=num_classes)

x=base_model.output

x = Flatten()(x)

x = Dense(units=512, activation='relu', kernel_regularizer=l2(weight_decay))(x)

# Layer 4
x = Dense(units=64, activation='relu', kernel_regularizer=l2(weight_decay))(x)

# Layer 5
output = Dense(units=num_classes, activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=output)
model.summary()




58892288/58889256 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
from keras.optimizers import SGD
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

**Data augmentation**   
Các ảnh trong tập training được xoay, miror,chỉnh sáng, và cuối cùng là normalize giá trị các pixel về khoảng [0,1]

In [0]:
from keras.preprocessing.image import ImageDataGenerator

def normalize(input):
  return input/255.
datagen=ImageDataGenerator(rotation_range=30,horizontal_flip=True,preprocessing_function=normalize,brightness_range=[0.8,1.25])

aug_gen=datagen.flow(x_train, y_train, batch_size=len(x_train),shuffle=False)

In [0]:
# train the model on the new data for a few epochs
model.fit_generator(aug_gen,epochs=200,steps_per_epoch=1)

Epoch 1/200
1/1 [==============================] - 10s 10s/step - loss: 2.3168 - acc: 0.1758
Epoch 2/200
1/1 [==============================] - 0s 194ms/step - loss: 2.0371 - acc: 0.4510
Epoch 3/200
1/1 [==============================] - 0s 201ms/step - loss: 1.8967 - acc: 0.6352
Epoch 4/200
1/1 [==============================] - 0s 194ms/step - loss: 1.7670 - acc: 0.6633
Epoch 5/200
1/1 [==============================] - 0s 192ms/step - loss: 1.6383 - acc: 0.6788
Epoch 6/200
1/1 [==============================] - 0s 188ms/step - loss: 1.5184 - acc: 0.7057
Epoch 7/200
1/1 [==============================] - 0s 202ms/step - loss: 1.4046 - acc: 0.7380
Epoch 8/200
1/1 [==============================] - 0s 199ms/step - loss: 1.3236 - acc: 0.7620
Epoch 9/200
1/1 [==============================] - 0s 194ms/step - loss: 1.2547 - acc: 0.7739
Epoch 10/200
1/1 [==============================] - 0s 201ms/step - loss: 1.1889 - acc: 0.7871
Epoch 11/200
1/1 [==============================] - 0s 370ms

Kiểm tra độ chính xác sau bước training các tầng fully-connected

In [0]:
model.evaluate(x_test/255,y_test/255)

183/183 [==============================] - 0s 486us/step


[0.1502167450599983, 0.8469945345420004]

In [0]:
for layer in base_model.layers:
    layer.trainable = True

In [0]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(aug_gen,epochs=100,steps_per_epoch=1)

Epoch 1/100
1/1 [==============================] - 16s 16s/step - loss: 0.2281 - acc: 0.9749
Epoch 2/100
1/1 [==============================] - 1s 584ms/step - loss: 0.2338 - acc: 0.9779
Epoch 3/100
1/1 [==============================] - 1s 604ms/step - loss: 0.2448 - acc: 0.9695
Epoch 4/100
1/1 [==============================] - 1s 591ms/step - loss: 0.3393 - acc: 0.9252
Epoch 5/100
1/1 [==============================] - 1s 590ms/step - loss: 1.2534 - acc: 0.7183
Epoch 6/100
1/1 [==============================] - 1s 586ms/step - loss: 3.6821 - acc: 0.5855
Epoch 7/100
1/1 [==============================] - 1s 595ms/step - loss: 1.0143 - acc: 0.6968
Epoch 8/100
1/1 [==============================] - 1s 585ms/step - loss: 0.8370 - acc: 0.7368
Epoch 9/100
1/1 [==============================] - 1s 593ms/step - loss: 0.4809 - acc: 0.8774
Epoch 10/100
1/1 [==============================] - 1s 591ms/step - loss: 0.4027 - acc: 0.9079
Epoch 11/100
1/1 [==============================] - 1s 604ms

Test model đã huấn luyện với dữ liệu test:

In [0]:
model.evaluate(x_test/255, y_test/255)

183/183 [==============================] - 1s 7ms/step


[0.15002270718741287, 0.9016393406795022]